In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

def transform_bronze_to_silver(table_name, catalog_schema="silver"):
    """
    Toma una tabla de Bronze, limpia nombres de columnas, 
    agrega metadata y la guarda en Silver.
    """
    print(f"Transformando {table_name} a Silver...")

In [0]:
from delta.tables import *
from pyspark.sql import functions as F

# 1. CONFIGURACIÓN
CATALOGO = "santig_120781"
ESQUEMA_BRONZE_OLD = f"{CATALOGO}.bronze"
ESQUEMA_BRONZE_NEW = f"{CATALOGO}.ingesta_nueva_bronze"
ESQUEMA_SILVER     = f"{CATALOGO}.silver"

spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")

tablas_claves = {
    "products": "product_id",
    "departments": "department_id",
    "aisles": "aisle_id",
    "orders": "order_id",
    "order_products__prior": "order_id", 
    "order_products__train": "order_id"
}

print(f"--- Iniciando Unificación con Limpieza de Duplicados ---")

for tabla, llave_primaria in tablas_claves.items():
    print(f"🔨 Procesando tabla: {tabla}...")

    # 2. LECTURA Y UNIFICACIÓN
    df_old = spark.read.table(f"{ESQUEMA_BRONZE_OLD}.{tabla}")
    
    if spark.catalog.tableExists(f"{ESQUEMA_BRONZE_NEW}.{tabla}"):
        df_new = spark.read.table(f"{ESQUEMA_BRONZE_NEW}.{tabla}")
        df_source = df_old.unionByName(df_new, allowMissingColumns=True)
    else:
        df_source = df_old

    # 3. LIMPIEZA DE COLUMNAS (Split de ';')
    for col_name in df_source.columns:
        if ";" in col_name:
            parts = F.split(F.col(col_name), ";")
            nuevos_nombres = col_name.split(";")
            df_source = df_source.withColumn(nuevos_nombres[0], parts.getItem(0).cast("int")) \
                                 .withColumn(nuevos_nombres[1], F.trim(parts.getItem(1))) \
                                 .drop(col_name)
        else:
            nombre_limpio = col_name.strip().replace(" ", "_").replace(";", "_")
            df_source = df_source.withColumnRenamed(col_name, nombre_limpio)

    # Nos quedamos solo con un registro por cada ID (el más reciente si fuera posible)
    df_source = df_source.dropDuplicates([llave_primaria])

    # 4. MERGE
    df_source = df_source.withColumn("_process_date", F.current_timestamp())

    if not spark.catalog.tableExists(f"{ESQUEMA_SILVER}.{tabla}"):
        df_source.write.format("delta").mode("overwrite").saveAsTable(f"{ESQUEMA_SILVER}.{tabla}")
        print(f"  Tabla {tabla} creada.")
    else:
        dt_silver = DeltaTable.forName(spark, f"{ESQUEMA_SILVER}.{tabla}")
        
        if tabla == "orders":
            columnas_validas = [c for c in df_source.columns if c != "order_date"]
            df_orders_final = df_source.select(*columnas_validas)
            
            dt_silver.alias("tgt").merge(
                df_orders_final.alias("src"),
                f"tgt.{llave_primaria} = src.{llave_primaria}"
            ).whenNotMatchedInsertAll() \
             .execute()
            print(f"   [Orders] Sincronizado sin duplicados.")
        else:
            dt_silver.alias("tgt").merge(
                df_source.alias("src"),
                f"tgt.{llave_primaria} = src.{llave_primaria}"
            ).whenMatchedUpdateAll() \
             .whenNotMatchedInsertAll() \
             .execute()
            print(f"  Merge completado para {tabla}.")

print("\n🚀 ¡Proceso terminado exitosamente!")

--- Iniciando Unificación con Limpieza de Duplicados ---
🔨 Procesando tabla: products...
   ✅ Merge completado para products.
🔨 Procesando tabla: departments...
   ✅ Merge completado para departments.
🔨 Procesando tabla: aisles...
   ✅ Merge completado para aisles.
🔨 Procesando tabla: orders...
   ✅ [Orders] Sincronizado sin duplicados.
🔨 Procesando tabla: order_products__prior...
   ✅ Merge completado para order_products__prior.
🔨 Procesando tabla: order_products__train...
   ✅ Merge completado para order_products__train.

🚀 ¡Proceso terminado exitosamente!


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS santig_120781.bronze")

DataFrame[]

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 1. Configuración de rutas
CATALOGO = "santig_120781"
ESQUEMA_BRONZE_OLD = f"{CATALOGO}.bronze"
ESQUEMA_BRONZE_NEW = f"{CATALOGO}.ingesta_nueva_bronze"
SILVER_ORDERS      = f"{CATALOGO}.silver.orders"

print("--- Iniciando Reconstrucción de Fechas (Carga Histórica + Nueva) ---")

# 2. LECTURA Y UNIFICACIÓN (Garantizamos traer los datos nuevos)
df_old = spark.read.table(f"{ESQUEMA_BRONZE_OLD}.orders")

if spark.catalog.tableExists(f"{ESQUEMA_BRONZE_NEW}.orders"):
    df_new = spark.read.table(f"{ESQUEMA_BRONZE_NEW}.orders")
    df_unificado = df_old.unionByName(df_new, allowMissingColumns=True)
    print(" Datos de ingesta nueva detectados para el cálculo de fechas.")
else:
    df_unificado = df_old
    print("ℹ Usando solo datos de Bronze original.")

# 3. Limpieza de nombres y eliminación de duplicados (Seguridad extra)
df_limpio = df_unificado.dropDuplicates(["order_id"])
for col in df_limpio.columns:
    df_limpio = df_limpio.withColumnRenamed(col, col.replace(";", "_"))

# 4. Localizar columna de días
col_dias = "days_since_prior_order" if "days_since_prior_order" in df_limpio.columns else [c for c in df_limpio.columns if "days_since" in c][0]

# 5. Lógica de Simulación de Fecha
# Usamos coalesce para que los nulos (primeras órdenes) cuenten como 0 días
df_temp = df_limpio.withColumn(
    "days_since_prior_num", 
    F.coalesce(F.col(col_dias).cast("int"), F.lit(0))
)

# Definimos la ventana por usuario para sumar los días en orden
window_user = Window.partitionBy("user_id").orderBy("order_number")

df_with_dates = df_temp.withColumn(
    "accumulated_days", 
    F.sum("days_since_prior_num").over(window_user)
).withColumn(
    "order_date", 
    F.date_sub(F.lit("2024-01-01"), F.col("accumulated_days").cast("int"))
)

# 6. Selección final y Guardado
# IMPORTANTE: Incluimos days_since_prior_order para que el MERGE de la Celda 2 no falle la próxima vez
df_final = df_with_dates.select(
    "order_id",
    "user_id",
    "order_number",
    "order_dow",
    "order_hour_of_day",
    "days_since_prior_order", # <-- Mantenemos esta columna viva
    "order_date",
    F.current_timestamp().alias("_process_date")
)

df_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(SILVER_ORDERS)

print(f" ¡Éxito! La tabla {SILVER_ORDERS} se ha actualizado con {df_final.count()} registros.")
display(df_final.sort("user_id", "order_number").limit(10))

--- Iniciando Reconstrucción de Fechas (Carga Histórica + Nueva) ---
✅ Datos de ingesta nueva detectados para el cálculo de fechas.
✅ ¡Éxito! La tabla santig_120781.silver.orders se ha actualizado con 1775743 registros.


order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_date,_process_date
2539329,1,1,2,8,null,2024-01-01,2026-02-15T03:24:55.295456Z
2398795,1,2,3,7,15.0,2023-12-17,2026-02-15T03:24:55.295456Z
473747,1,3,3,12,21.0,2023-11-26,2026-02-15T03:24:55.295456Z
2254736,1,4,4,7,29.0,2023-10-28,2026-02-15T03:24:55.295456Z
431534,1,5,4,15,28.0,2023-09-30,2026-02-15T03:24:55.295456Z
3367565,1,6,2,7,19.0,2023-09-11,2026-02-15T03:24:55.295456Z
550135,1,7,1,9,20.0,2023-08-22,2026-02-15T03:24:55.295456Z
3108588,1,8,1,14,14.0,2023-08-08,2026-02-15T03:24:55.295456Z
2295261,1,9,1,16,0.0,2023-08-08,2026-02-15T03:24:55.295456Z
2550362,1,10,4,8,30.0,2023-07-09,2026-02-15T03:24:55.295456Z
